In [2]:
# Import Libraries
%matplotlib inline
import re
import matplotlib.pyplot as plt
import numpy as np
import Bio.SeqIO as SeqIO
import time
from sklearn.cluster import KMeans, k_means

In [3]:
# Extract Data
f = open("sequence.gb","r")
out = f.read()
f.close()

j = 0
dict = []
for i, record in enumerate(SeqIO.parse("sequence.gb", "genbank")):
    if all (key in record.features[0].qualifiers for key in ('host','country','collection_date')):
        j+=1
        
        host = record.features[0].qualifiers['host'][0]
        host = host.split(';')[0]
        
        location = record.features[0].qualifiers['country'][0]
        location = location.split(':')[0]
        
        year = record.features[0].qualifiers['collection_date'][0]
        year = year.split('-')[-1]
        
        dict.append(
            {
                'id'          :record.id,
                'name'        :record.name,
                'description' :record.description,
                'host'        :host,
                'location'    :location,
                'year'        :year,
                'seq'         :record.seq,
                'url'         :'http://www.ncbi.nlm.nih.gov/nuccore/'+record.id
            }
        )

In [ ]:
for row in dict:
    print (row['id'] + ' ' + row['url'])

In [4]:
def match_score(A,B):
    if A == B: return 1
    else: return -1

In [5]:
def print_table(S,header):
    print(header)
    for i in range(0,len(S)):
        print(S[i])
    print('\n')

In [51]:
# Global Alignment
def complete_global_alignment(v,w):
    m = len(v)
    n = len(w)

    # Backtrack enum
    R_UP = 1
    R_LEFT = 2
    R_DIAG = 3

    d = -1
    S = [[0 for x in range(len(v)+1)] for y in range(len(w)+1)]
    B = [[0 for x in range(len(v)+1)] for y in range(len(w)+1)]

    for i in range(1,len(w)+1):
        S[i][0] = d*i
        B[i][0] = R_UP
    for j in range(1,len(v)+1):
        S[0][j] = d*j
        B[0][j] = R_LEFT
    for i in range(1,len(w)+1):
        for j in range(1,len(v)+1):
            match_score = 1 if v[j-1] == w[i-1] else -1
            Match = S[i-1][j-1] + match_score
            Insert = S[i-1][j] + d
            Delete = S[i][j-1] + d
            S[i][j] = max(Match, Insert, Delete)
            if S[i][j] == Match:
                B[i][j] = R_DIAG
            elif S[i][j] == Insert:
                B[i][j] = R_UP
            elif S[i][j] == Delete:
                B[i][j] = R_LEFT
    
    # Print resulting table
    #print_table(S,'Resulting Table')
    
    # Print backtrack matrix
    #print_table(B,'Backtrack Matrix')
    
    # Print resulting string
    #print('Backtrack Process')
    vr = ""
    wr = ""
    i = n
    j = m
    k = 0
    while B[i][j] != 0:
        if B[i][j] == R_DIAG:
            vr = v[j-1] + vr
            wr = w[i-1] + wr
            #print(i,j,'DIAG')
            i = i-1
            j = j-1
        elif B[i][j] == R_LEFT:
            wr = '-' + wr
            vr = v[j-1] + vr
            #print(i,j,'LEFT')
            j = j-1
        elif B[i][j] == R_UP:
            vr = '-' + vr
            wr = w[i-1] + wr
            #print(i,j,'UP')
            i = i-1

    #print('\n')
    
    #print('Global Alignment')
    #print('Length v:')
    #print(len(vr))
    #print('Length w:')
    #print(len(wr))
    #print(vr)
    #print(wr)
    #print('Score:')
    #print(S[len(w)][len(v)])
    return S[len(w)][len(v)]

In [50]:
# Global Alignment
def global_alignment(v,w):
    m = len(v)
    n = len(w)

    d = -1
    S = [[0 for x in range(len(v)+1)] for y in range(len(w)+1)]

    for i in range(1,len(w)+1):
        S[i][0] = d*i
    for j in range(1,len(v)+1):
        S[0][j] = d*j
    for i in range(1,len(w)+1):
        for j in range(1,len(v)+1):
            match_score = 1 if v[j-1] == w[i-1] else -1
            Match = S[i-1][j-1] + match_score
            Insert = S[i-1][j] + d
            Delete = S[i][j-1] + d
            S[i][j] = max(Match, Insert, Delete)
                
    return S[len(w)][len(v)]

In [49]:
def complete_local_alignment(v,w):
    m = len(v)
    n = len(w)

    # Backtrack enum
    R_UP = 1
    R_LEFT = 2
    R_DIAG = 3

    d = -1
    max_val = -1
    max_row = 0
    max_col = 0
    S = [[0 for x in range(len(v)+1)] for y in range(len(w)+1)]
    B = [[0 for x in range(len(v)+1)] for y in range(len(w)+1)]

    for i in range(1,len(w)+1):
        for j in range(1,len(v)+1):
            match_score = 1 if v[j-1] == w[i-1] else -1
            Match = S[i-1][j-1] + match_score
            Insert = S[i-1][j] + d
            Delete = S[i][j-1] + d
            S[i][j] = max(Match, Insert, Delete, 0)
            if S[i][j] > max_val:
                max_val = S[i][j]
                max_row = i
                max_col = j
            if S[i][j] == 0:
                continue
            elif S[i][j] == Match:
                B[i][j] = R_DIAG
            elif S[i][j] == Insert:
                B[i][j] = R_UP
            elif S[i][j] == Delete:
                B[i][j] = R_LEFT

    # Print resulting table
    #print_table(S,'Resulting Table')
          
    # Print backtrack matrix
    #print_table(B,'Backtrack Matrix')
    
    # Print resulting string
    #print('Backtrack Process')
    vr = ""
    wr = ""
    i = max_row
    j = max_col
    #print('Max Row',max_row,'Max Col',max_col)
    while B[i][j] > 0:
        if B[i][j] == R_DIAG:
            vr = v[j-1] + vr
            wr = w[i-1] + wr
            #print(i,j,'DIAG')
            i = i-1
            j = j-1
        elif B[i][j] == R_LEFT:
            #wr = '-' + wr
            vr = v[j-1] + vr
            #print(i,j,'LEFT')
            j = j-1
        elif B[i][j] == R_UP:
            #vr = '-' + vr
            wr = w[i-1] + wr
            #print(i,j,'UP')
            i = i-1

    #print('\n')
    
    #print('Local Alignment')
    #print('Length v:')
    #print(len(vr))
    #print('Length w:')
    #print(len(wr))
    #print(vr)
    #print(wr)
    #vx = vr
    #wx = wr
    #print('Score:')
    #print(max_val)
    #return vr,wr
    return max_val

In [46]:
def local_alignment(v,w):
    m = len(v)
    n = len(w)

    d = -1
    max_val = -1
    max_row = 0
    max_col = 0
    S = [[0 for x in range(len(v)+1)] for y in range(len(w)+1)]

    for i in range(1,len(w)+1):
        for j in range(1,len(v)+1):
            match_score = 1 if v[j-1] == w[i-1] else -1
            Match = S[i-1][j-1] + match_score
            Insert = S[i-1][j] + d
            Delete = S[i][j-1] + d
            S[i][j] = max(Match, Insert, Delete, 0)
            if S[i][j] > max_val:
                max_val = S[i][j]

    return max_val

In [10]:
indonesian_virus = []
for row in dict:
    if row['location'] == 'Indonesia':
        indonesian_virus.append(row)

In [33]:
#get centroid
seq_len_kmeans = [[len(row['seq'])] for row in dict]
seq_centroid, seq_cluster, seq_cluster_error = k_means(seq_len_kmeans, n_clusters=2)
seq_centroid

array([[ 10614.50588235],
       [   902.625     ]])

In [31]:
#get threshold: the average of centroids
threshold = (seq_centroid[0] + seq_centroid[1])/2
threshold

array([ 5758.56544118])

In [35]:
len(indonesian_virus[0]['seq'])

1148

In [36]:
len(dict[2]['seq'])

10735

In [45]:
#individual check
start_time = time.time() # execution time
score = local_alignment(indonesian_virus[0]['seq'],dict[2]['seq'])
print(score)
print("Time elapsed: %s seconds" % (time.time() - start_time))

556
Time elapsed: 22.553848028182983 seconds


In [48]:
#individual check
start_time = time.time() # execution time
v = str(indonesian_virus[0]['seq'])
w = str(dict[2]['seq'])
score = local_alignment(v,w)
print(score)
print("Time elapsed: %s seconds" % (time.time() - start_time))

556
Time elapsed: 13.05771517753601 seconds


In [40]:
#group check
start_time = time.time() # execution time
score = []
i = 1
for ind in indonesian_virus:
    for row in dict:
        print(i)
        virus_time = time.time();
        if ind['id'] == row['id']:
            continue
        print(ind['id'] , ' vs ' , row['id'])
        i += 1
        s = ""
        if len(row['seq']) > threshold:
            print('Local')
            s = local_alignment(ind['seq'],row['seq'])
        else:
            print('Global')
            s = global_alignment(ind['seq'],row['seq'])
        score.append(s)
        print('Score: ' , s)
        print("Time elapsed: %s seconds" % (time.time() - virus_time))
        if i > 1: #Flow control to test
            break
print("Total time elapsed: %s seconds" % (time.time() - start_time))

1
KU179098.1  vs  KX101066.1
Local
Score:  290
Time elapsed: 23.44686508178711 seconds
2
KF258813.1  vs  KX101066.1
Local
Score:  80
Time elapsed: 7.871891021728516 seconds
Total time elapsed: 31.319257020950317 seconds
